In [ ]:
import pandas as pd
from clingo import Control
import ast
from matching_utils import (
    get_invalid_matches, 
    check_mentor_capacity,
    check_day_conflicts,
    check_multiple_days
)

students_df = pd.read_csv('../DATASETS/studenten.csv')
mentors_df = pd.read_csv('../DATASETS/mentoren.csv')

education_mapping = {
    'Associate': 1,
    'Bachelor': 2, 
    'Master': 3,
    'PhD': 4
}

n_mentors = 3

def parse_literal_list(raw):
    if isinstance(raw, str):
        try:
            return ast.literal_eval(raw)
        except (ValueError, SyntaxError):
            return []
    return raw if isinstance(raw, list) else []

def normalize_token(text):
    if not isinstance(text, str):
        return ""
    return text.lower().replace(' ', '_')

students_cache = []
student_lookup = {}
for idx, row in students_df.iterrows():
    student_id = f"s{idx}"
    subject_atom = normalize_token(row['Onderwerp'])
    entry = {
        'id': student_id,
        'education_level': education_mapping[row['Opleidingsniveau']],
        'subject_atom': subject_atom,
        'data': {
            'voornaam': row['Voornaam'],
            'achternaam': row['Achternaam'],
            'opleidingsniveau': row['Opleidingsniveau'],
            'onderwerp': row['Onderwerp'],
        },
    }
    students_cache.append(entry)
    student_lookup[student_id] = entry

mentors_cache = []
mentor_lookup = {}
for idx, row in mentors_df.iterrows():
    mentor_id = f"m{idx}"
    subjects = [normalize_token(subject) for subject in parse_literal_list(row['Onderwerpen'])]
    availability = [day.lower() for day in parse_literal_list(row['Beschikbaarheid'])]
    entry = {
        'id': mentor_id,
        'education_level': education_mapping[row['Opleidingsniveau']],
        'subjects': subjects,
        'availability': availability,
        'max_students': row['Max_Studenten'],
        'data': {
            'voornaam': row['Voornaam'],
            'achternaam': row['Achternaam'],
            'opleidingsniveau': row['Opleidingsniveau'],
        },
    }
    mentors_cache.append(entry)
    mentor_lookup[mentor_id] = entry

def generate_asp_facts():
    facts = []
    days = set()
    
    for student in students_cache:
        facts.append(f"student({student['id']}).")
        facts.append(f"education({student['id']}, {student['education_level']}).")
        facts.append(f"expertise({student['id']}, {student['subject_atom']}).")
    
    for mentor in mentors_cache:
        facts.append(f"mentor({mentor['id']}).")
        facts.append(f"education({mentor['id']}, {mentor['education_level']}).")
        
        for subject in mentor['subjects']:
            facts.append(f"expertise({mentor['id']}, {subject}).")
        
        facts.append(f"max_students({mentor['id']}, {mentor['max_students']}).")
        
        for day in mentor['availability']:
            facts.append(f"availability({mentor['id']}, {day}).")
            days.add(day)
    
    for day in sorted(days):
        facts.append(f"day({day}).")
    
    return "\n".join(facts)

asp_facts = generate_asp_facts()

ctl = Control()
ctl.configuration.solve.models = 1
ctl.add("base", [], f"""
#const req_mentors = {n_mentors}.

{asp_facts}

candidate(S, M, Day) :-
    student(S),
    mentor(M),
    expertise(S, Subj),
    expertise(M, Subj),
    education(S, ES),
    education(M, EM),
    EM > ES,
    availability(M, Day).

possible_day(S, Day) :- candidate(S, M, Day).

1 {{ match_day(S, Day) : possible_day(S, Day) }} 1 :- student(S).

{{ match(S, M, Day) : candidate(S, M, Day) }} :- match_day(S, Day).

:- match_day(S, Day), #count {{ M : match(S, M, Day) }} < req_mentors.
:- match_day(S, Day), #count {{ M : match(S, M, Day) }} > req_mentors.

:- match(S, M, Day), not match_day(S, Day).

:- mentor(M), max_students(M, Max), #count {{ S, Day : match(S, M, Day) }} > Max.

#maximize {{ 1, S : match_day(S, Day) }}.

#show match/3.

""")

ctl.ground([("base", [])])

def make_matches():
    matches = []

    def collect_matches(model):
        nonlocal matches
        grouped = {}

        for symbol in model.symbols(shown=True):
            if symbol.name == "match":
                student_id = symbol.arguments[0].name
                mentor_id = symbol.arguments[1].name
                day = symbol.arguments[2].name

                key = (student_id, day)
                grouped.setdefault(key, []).append(mentor_id)

        matches = []
        for (student_id, day), mentor_ids in grouped.items():
            student_entry = student_lookup[student_id]
            student_data = student_entry['data'].copy()

            mentors_data = []
            for mentor_id in sorted(mentor_ids):
                mentor_entry = mentor_lookup[mentor_id]
                mentors_data.append(mentor_entry['data'].copy())

            matches.append((student_data, mentors_data, day))

    result = ctl.solve(on_model=collect_matches)
    return matches if result.satisfiable else []



# Usage
matches = make_matches()
print(f"Total matched students: {len(matches)} (each with {n_mentors} mentors)")
# for student, mentor, day in matches[:5]:
#     print(f"{student['voornaam']} {student['achternaam']} -> {mentor['voornaam']} {mentor['achternaam']} on {day.capitalize()}")

def export_matches_to_csv(matches, filename='../DATASETS/matches.csv'):
    if not matches:
        print("No matches to export")
        return

    rows = []
    for student, mentors, day in matches:
        row = {
            'Student': f"{student['voornaam']} {student['achternaam']}",
            'Day': day.capitalize(),
            'Mentors': "; ".join(f"{m['voornaam']} {m['achternaam']}" for m in mentors),
        }
        for idx, mentor in enumerate(mentors, start=1):
            row[f'Mentor{idx}'] = f"{mentor['voornaam']} {mentor['achternaam']}"
        rows.append(row)

    matches_df = pd.DataFrame(rows)
    matches_df.to_csv(filename, index=False)
    print(f"Exported {len(matches)} matches to {filename}")
    return matches_df

# Export the matches
matches_df = export_matches_to_csv(matches)

if matches_df is not None:
    print("\nPreview of exported matches:")
    print(matches_df.head())

Total matched students: 100 (each with 3 mentors)
Exported 100 matches to ../DATASETS/matches.csv

Preview of exported matches:
          Student        Day  \
0   Micha Claesdr    Dinsdag   
1   David Bronder  Donderdag   
2   Tijs Strijker    Maandag   
3  Niek Tillmanno    Vrijdag   
4  Aiden Kathagen    Vrijdag   

                                             Mentors            Mentor1  \
0  Noëlle Jorlink; Chloë van der Klein; Quinty va...     Noëlle Jorlink   
1    Alicia de Vries; Luke Mosley; Livia van Luyssel    Alicia de Vries   
2  Amélie Kort; Jip Heribert van Laon; Amélie Garret        Amélie Kort   
3  Ceylin Beourgeois; Jason Driessen; Felix van d...  Ceylin Beourgeois   
4  Nikki van de Wiel; Benjamin van der Flaas; Loï...  Nikki van de Wiel   

                  Mentor2             Mentor3  
0     Chloë van der Klein  Quinty van Enschot  
1             Luke Mosley   Livia van Luyssel  
2   Jip Heribert van Laon       Amélie Garret  
3          Jason Driessen   Felix va

In [ ]:
def debug_matching():
    matches = make_matches()
    print(f"Total matches found: {len(matches)}")
    
    if matches:
        print("\nSample matches:")
        for student, mentors, day in matches[:5]:
            print(f"  {student['voornaam']} {student['achternaam']} ({student['opleidingsniveau']}, {student['onderwerp']})")
            print(f"  Day: {day.capitalize()}")
            for mentor in mentors:
                print(f"    -> {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']})")
            print()
    
    flat_matches = [
        (student, mentor, day)
        for student, mentors, day in matches
        for mentor in mentors
    ]
    
    print("\n=== Validation Results ===")
    
    invalid_matches = get_invalid_matches(flat_matches)
    if invalid_matches:
        print(f"[FAIL] Invalid matches: {len(invalid_matches)}")
        for student, mentor, day in invalid_matches[:3]:
            print(f"   - {student['voornaam']} {student['achternaam']} -> {mentor['voornaam']} {mentor['achternaam']} on {day}")
    else:
        print("[PASS] All matches are valid")
    
    over_capacity = check_mentor_capacity(flat_matches)
    if over_capacity:
        print(f"[FAIL] Mentors over capacity:")
        for item in over_capacity:
            print(f"   - {item['mentor']}: {item['matched']} matched (max: {item['max']})")
    else:
        print("[PASS] All mentors within capacity")
    
    day_conflicts = check_day_conflicts(flat_matches)
    if day_conflicts:
        print(f"[FAIL] Day conflicts found:")
        for conflict in day_conflicts:
            print(f"   - {conflict['type'].capitalize()} {conflict['name']} has {conflict['count']} matches on {conflict['day']}")
    else:
        print("[PASS] No day conflicts")
    
    multiple_days = check_multiple_days(flat_matches)
    if multiple_days:
        print(f"[FAIL] Students matched on multiple days:")
        for item in multiple_days:
            print(f"   - {item['student']}: {', '.join(item['days'])}")
    else:
        print("[PASS] No students matched on multiple days")

debug_matching()

Total matches found: 100

Sample matches:
  Micha Claesdr (Bachelor, Creative Digital Innovation)


TypeError: list indices must be integers or slices, not str